In [1]:
submission_files = [
    "./submission_5h1l_tile_4_5_overlaps_0_0_topk_1_minscore_002_topn_10_use_gf_crop_010.csv",
    "./submission_5heads_2layers_ckpt_ep2_it4000_tile_5_overlaps_0_topk_1_minscore_001_use_gf.csv",
]

submission_name = "032760_036590_5h1l_tile_4_5_overlaps_0_0_topk_1_minscore_002_topn_10_use_gf_crop_010_and_5heads_2layers_ckpt_ep2_it4000_tile_5_overlaps_0_topk_1_minscore_001_use_gf"

In [2]:
from collections import defaultdict
import csv
import pandas as pd
import ast

# Load submission files into DataFrames
dfs = [
    pd.read_csv(file, converters={"species_ids": ast.literal_eval})
    for file in submission_files
]

predictions: dict[str, set] = defaultdict(set)
for df in dfs:
    for quadrat_id, species_ids in zip(df["quadrat_id"], df["species_ids"]):
        for species_id in species_ids:
            predictions[quadrat_id].add(species_id)

df_run = pd.DataFrame(
    list(predictions.items()),
    columns=[
        "quadrat_id",
        "species_ids",
    ],
)
df_run["species_ids"] = df_run["species_ids"].apply(list).apply(str)
df_run.to_csv(
    f"unions_{submission_name}.csv",
    sep=",",
    index=False,
    quoting=csv.QUOTE_ALL,
)

In [3]:
# Compute the intersection of species_ids for each quadrat_id
intersections: dict[str, set] = defaultdict(set)

# Initialize intersections with the species_ids from the first DataFrame
for quadrat_id, species_ids in zip(dfs[0]["quadrat_id"], dfs[0]["species_ids"]):
    intersections[quadrat_id] = set(species_ids)

# Perform intersection with the species_ids from the remaining DataFrames
for df in dfs[1:]:
    for quadrat_id, species_ids in zip(df["quadrat_id"], df["species_ids"]):
        intersections[quadrat_id] &= set(species_ids)

# Convert the intersections dictionary to a DataFrame
df_intersection = pd.DataFrame(
    list(intersections.items()),
    columns=[
        "quadrat_id",
        "species_ids",
    ],
)
df_intersection["species_ids"] = df_intersection["species_ids"].apply(list).apply(str)
df_intersection.to_csv(
    f"intersections_{submission_name}.csv",
    sep=",",
    index=False,
    quoting=csv.QUOTE_ALL,
)